In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 M

In [ ]:
pip install datasets

In [ ]:
pip install accelerate

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:00


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [ ]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "test_detection")

print(data.keys())

data_de_en = data['test_detection_de_en'].to_pandas()
data_en_de = data['test_detection_en_de'].to_pandas()
data_fr_en = data['test_detection_fr_en'].to_pandas()
data_en_fr = data['test_detection_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating test_detection_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['test_detection_de_en', 'test_detection_en_de', 'test_detection_fr_en', 'test_detection_en_fr'])


,id,langpair,source,hyp1,hyp2
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In developed countries, offering luxury guesth..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice elsewhere in the UK, b..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"“Based on this fossil, we can say that the sep...","“Based on this fossil, we can say that the sep..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","An Avenger class anti-mining ship, the ship wa...","An Avenger class anti-mining ship, the ship wa..."


,id,langpair,source,hyp1,hyp2
0,0,en-de,He graduated from the College of Arts & Scienc...,Er absolvierte 1950 das College of Arts & Scie...,Er absolvierte 1950 das College of Arts & Scie...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten der kleineren Inseln sind unabhäng...,"Die meisten sind unabhängige Nationen, oder mi..."
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die immer...","Aber es gibt viele Dinge über Vögel, die wie e..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...",Die neuseeländische Polizei hatte Schwierigkei...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","""Das Plateau von Giza oder """"Nekropolis von Gi...","""Das Plateau von Giza oder """"Nekropolis von Gi..."


In [ ]:
#load the trial data for all language pairs
trial_ds = load_dataset("Eloquent/HalluciGen-Translation", name="trial")
print(trial_ds.keys())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

dict_keys(['trial_de_en', 'trial_en_de', 'trial_fr_en', 'trial_en_fr'])


In [ ]:
trial_ds_de_en = trial_ds['trial_de_en'].to_pandas()
trial_ds_en_de = trial_ds['trial_en_de'].to_pandas()
trial_ds_fr_en = trial_ds['trial_fr_en'].to_pandas()
trial_ds_en_fr = trial_ds['trial_en_fr'].to_pandas()

display(trial_ds_de_en.head())
display(trial_ds_en_de.head())

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems ge...,"Die Tage im Sommer können zu Problemen führen,...",Die sehr langen Tage im Sommer können zu Probl...,addition,hyp2
1,1,en-de,The vertical clearance under the bridge is 15 ...,Der vertikale Abstand unter der Brücke beträgt...,Der vertikale Abstand unter der Brücke beträgt...,date,hyp1
2,2,en-de,William Llewelyn Williams known as Llewelyn Wi...,"William Williams, bekannt als Llewelyn William...","Williajm Llewelyn Williams, bekannt als Llewel...",named-entity,hyp2
3,3,en-de,The single was released on Radio Airplay in It...,Die Single wurde am 12. Oktober 212 in Italien...,Die Single wurde am 12. Oktober 2012 in Italie...,number,hyp1
4,4,en-de,Water is spilling over the levee in a section ...,Wasser fließt über den Deich in einem Abschnit...,Wasser fließt über die Leibe in einem Abschnit...,natural,hyp2


## Load Model

In [ ]:
#define the Model
model = "google/gemma-7b-it"
# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt 1

In [ ]:
def generate_label(source, hyp1, hyp2, translation_language):

  answer_format = {"label": ""}

  # prompt for translation generation task
  user_prompt = f'''<start_of_turn>user
    You are a researcher investigating a new phenomenon. You have gathered data (source) and
    formulated two competing hypotheses (hyp1 and hyp2) to explain it.
    Identify the hypothesis that contradicts the information provided in the given source.
    Provide the result in the following format: {answer_format}. ONLY ANSWER IN THAT FORMAT!

    source: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    <end_of_turn>

    Result:
    <start_of_turn>model
    '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [££
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  label = outputs[0]["generated_text"][len(prompt):]
  print("label: ", label)

  return label

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    print("result_text: ", result_text)
    if result_text.find(": '") == -1 and result_text.find("'}") == -1:
      result_text = result_text.replace(": ", ": '").replace("}", "'}")
    print("result_text after replace: ", result_text)
    result_text = result_text.lower()
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

### Test

In [ ]:
source = trial_ds_de_en.iloc[0]["source"]
hyp1 = trial_ds_de_en.iloc[0]["hyp1"]
hyp2 = trial_ds_de_en.iloc[0]["hyp2"]
hallucination = generate_label(source, hyp1, hyp2, "English")
json_data = extract_json_data(hallucination)
print(json_data)


**Result:** {'label': 'Hyp2'}

The source information states that wandering ants march at night and rest during the day. Therefore, Hyp1 is consistent with the source information, while Hyp2 contradicts it.
{'label': 'Hyp2'}


In [ ]:
trial_ds_de_en.iloc[0]["label"]


'hyp2'

# Test with Trial dataset

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = generate_label(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/trial_results_gemma_de_en2.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")

label:  Result: {'label': hyp2}

The text states that "Wanderameisen marschieren bei Nacht und machen tagsüber Rast", which translates to "wandering ants march at night and rest during the day". Therefore, hyp1 is the correct hypothesis that aligns with the information provided in the source, while hyp2 contradicts it.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp1}

The source text states that the injury occurred on August 5th, while hyp1 incorrectly states that it occurred on December 5th. Therefore, hyp1 contradicts the information provided in the source.
result_text:  {'label': hyp1}
result_text after replace:  {'label': 'hyp1'}
label:  {'label': 'hyp2'}

The source text states that the auditor caught the male developer cheating, therefore hyp2 contradicts the information provided in the source.
result_text:  {'label': 'hyp2'}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The source information states that

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/trial_results_gemma_de_en2.csv")

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/trial_results_gemma_de_en2.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp2
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.20      0.33         5
        hyp2       0.56      1.00      0.71         5

    accuracy                           0.60        10
   macro avg       0.78      0.60      0.52        10
weighted avg       0.78      0.60      0.52        10



# DE - EN

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_de_en.csv"
skipped_rows = save_trial_results(data_de_en, csv_filename, "English")

label:  {'label': hyp2}

The source information states that "in Ländern" (translated to "in countries") the offering of luxury guesthouses has become an art form. Therefore, hyp1 is the hypothesis that aligns with the information provided in the source, while hyp2 does not.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The text describes the practice of publishing photos in the UK, specifically in Scotland. It states that this practice is common in the UK but not in Scotland. Therefore, the hypothesis that contradicts the information provided in the source is hyp2.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': 'hyp2'}

The source text mentions molecular evidence, while hyp2 refers to molecular evidence. Therefore, hyp2 contradicts the information provided in the source.
result_text:  {'label': 'hyp2'}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The text stat

# EN - DE

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_en_de.csv"
skipped_rows = save_trial_results(data_en_de, csv_filename, "German")

label:  Result: {'hyp2': ''}

The text describes a person who graduated from the College of Arts & Sciences in 1950 and was a significant donor to that institution. The text does not mention any information about the person's nationality, therefore I cannot provide the answer for hyp2.
result_text:  {'hyp2': ''}
result_text after replace:  {'hyp2': ''}
Skipping row 0 due to error in data extraction.
label:  {'label': hyp2}

The source information states that most of the smaller islands are independent nations, or associated with France, and known as luxury beach resorts. Therefore, the hypothesis that contradicts this information is hyp2.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The source information states that there are many things about birds that look like a dinosaur. Hyp1 is the exact translation of the source information, while hyp2 is an incorrect interpretation of the source information.
result_text:  {'label': hyp2}


# FR - EN

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_fr_en.csv"
skipped_rows = save_trial_results(data_fr_en, csv_filename, "English")
data_fr_en

label:  Result: {'label': 'hyp2'}

The text describes Martelly appointing a new Electoral Council (CEP) composed of nine members. However, the text does not mention a Provisional Electoral Council (CEP). Therefore, the hypothesis that contradicts the information provided in the source is hyp2.
result_text:  {'label': 'hyp2'}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The source information states that pronunciation is relatively easy in Italian, as most words are pronounced exactly as they are written. Therefore, hyp2 contradicts the information provided in the source.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The source text describes the emphasis of mysticism on the direct personal experience of a unique state of consciousness. However, hyp1 emphasizes the direct personal experience of a unique state of consciousness in the case of a peaceful, intuitive, happy, or even ecstatic personality. Therefo

,id,langpair,source,hyp1,hyp2
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has appointed a new Electoral Council...,Martelly has appointed a new Provisional Elect...
1,1,fr-en,"est relativement facile en italien, car la pl...","is relatively easy in Italian, as most words a...","Pronunciation is relatively easy in Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern Rock had applied for aid due to its e...,Northern had applied for aid due to its exposu...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the new population adapts to its..."
...,...,...,...,...,...
95,95,fr-en,"La couleur de capuchon varie, de marron à jaun...",The cap colorlessness varies from brown to yel...,"The cap color varies from brown to yellow, oft..."
96,96,fr-en,La rivière Arieşul Mare est un affluent de la ...,The Arieşul Mare River is a distributary of th...,The Arieşul Mare River is a tributary of the r...
97,97,fr-en,"Born est né en Allemagne de parents juifs, le ...","Born was born in Germany to Jewish parents, th...","Born in Germany was born to Jewish parent, the..."
98,98,fr-en,"À proximité se trouve la rivière Wallowa, un a...","Nearby is the Wallowa River, a distributary of...","Nearby is the Wallowa River, a tributary of th..."


# EN - FR

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_en_fr.csv"
skipped_rows = save_trial_results(data_en_fr, csv_filename, "French")

label:  Result: {'label': hyp2}

The source information states that Bobek was born in the Croatian capital, which is Zagreb. Therefore, the hypothesis that contradicts the information provided in the source is hyp2.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'label': hyp2}

The source information states that using GPS navigation apps is the easiest and most convenient way to navigate when out of your home country. Therefore, hyp1 is the hypothesis that contradicts the information provided in the source.
result_text:  {'label': hyp2}
result_text after replace:  {'label': 'hyp2'}
label:  {'Result': hyp2}

The source states that the effects of catastrophizing were conditional of regular weekly PA sessions, not depression and fear. Therefore, hyp1 is the hypothesis that contradicts the information provided in the source.
result_text:  {'Result': hyp2}
result_text after replace:  {'Result': 'hyp2'}
Skipping row 2 due to error in data extraction.
lab

## Convert to comma-separated CSV

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_de_en_final.csv"

df = pd.read_csv(csv_filename, sep=";")



In [ ]:
df.head()

,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp1


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_en_de_final.csv"

df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,pred_label
0,0,NaN
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_en_fr_final.csv"

df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp1
3,3,hyp2
4,4,hyp1


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma_newPrompt/results_gemma_fr_en_final.csv"

df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)
